## Define Credentials

In [136]:
server = 'https://ci.thuenen.de/rest/v1/'
anonkey = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.ewogICJyb2xlIjogImFub24iLAogICJpc3MiOiAiVEZNIiwKICAiaWF0IjogMTczOTkxOTYwMCwKICAiZXhwIjogMTg5NzY4NjAwMAp9.L28Sk6wzRLoUh1wLz_TjeY_rtUp3UX3-6UttadUEoC0'

## Import the necessary packages

In [137]:
import os

# install unittest package if not installed
try:
    import unittest
except ImportError:
    os.system('pip install unittest'
    'import unittest')

# install json package if not installed
try:
    import json
except ImportError:
    os.system('pip install json')
    import json

# install requests package if not installed
try:
    import requests
except ImportError:
    os.system('pip install requests')
    import requests

# install supabase package if not installed
try:
    from supabase import create_client, Client
except ImportError:
    os.system('pip install supabase')
    from supabase import create_client, Client

from colorama import Fore, Style

# Check Server Status

In [138]:
# Define the endpoint and headers
endpoint = f"{server}"
headers = {
    "apikey": f"{anonkey}",
    "Content-Type": "application/json"
}

# Example: Simulate a GET request
response = requests.get(endpoint, headers=headers)

# Check if the Supabase API is running
if response.status_code == 200:
    print(Fore.GREEN + "Supabase API is running.")
else:
    print(Fore.RED + f"Supabase API is not running. Status Code: {response.status_code}")

Supabase API is running.


# Table definitions

Array of tables with expected results for each test case.

In [157]:
tablesToTest = [
    {
        "tableName": "notExistingTable",
        "schema": "public",
        "columns": [
            "*"
        ],
        "exceptCode": 404,
        "exeptInsertCode": 404
    },
    {
        "tableName": "organizations",
        "schema": "public",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "schemas",
        "schema": "public",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "troop",
        "schema": "public",
        "columns": [
            "*"
        ],
        "exceptCode": 401,
        "exeptInsertCode": 401,
    },
    {
        "tableName": "records",
        "schema": "inventory",
        "columns": [
            "*"
        ],
        "exceptCode": 406,
        "exeptInsertCode": 406,
    },
    {
        "tableName": "record_changes",
        "schema": "inventory",
        "columns": [
            "*"
        ],
        "exceptCode": 406,
        "exeptInsertCode": 406,
    },
    {
        "tableName": "edges",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0,
    },
    {
        "tableName": "plot_landmark",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "tree_coordinates",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "plot_coordinates",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "subplots_relative_position",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 200,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 0
    },
    {
        "tableName": "cluster",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 206,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 65228
    },
    {
        "tableName": "plot",
        "schema": "inventory_archive",
        "columns": [
            "*"
        ],
        "exceptCode": 206,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 260912
    },
    {
        "tableName": "lookup_tree_species",
        "schema": "lookup",
        "columns": [
            "*"
        ],
        "exceptCode": 206,
        "exeptInsertCode": 401,
        "expected_response_type": list,
        "expected_length": 125
    }
]

# Test

Loop through the list of Tables and test for access to the data, type and length of the response.

In [159]:
class TestTableAccessibility(unittest.TestCase):
    def test_table_accessibility(self):

        for table in tablesToTest:

            #print('-' * 20 + table["tableName"] + '-' * 20)

            table_name = table["tableName"]
            url = f"{endpoint}{table_name}?limit=10"

            test_headers = headers.copy()

            # Add Schema
            test_headers["Accept-Profile"] = f"{table['schema']}"

            # Add Count
            if table.get("expected_length", None) is not None:
                test_headers["Prefer"] = "count=exact"

            #print(url)
            
            # Perform a GET request
            table_response = requests.get(url, headers=test_headers)
            
            # Check if the table is accessible
            actual_status_code = table_response.status_code
            expected_status_code = table["exceptCode"]
            expected_response_type = table.get("expected_response_type", None)
            expected_length = table.get("expected_length", None)

            # Get Count
            count = table_response.headers.get("Content-Range", None)
            
            try:
                self.assertEqual(actual_status_code, expected_status_code)

                print(Fore.GREEN + f"PASS: Table '{table_name}' status code matches expected." + Style.RESET_ALL)
            except AssertionError:
                print(Fore.RED + f"FAIL: Table '{table_name}' status code does not match expected." + Style.RESET_ALL)
                print(Fore.YELLOW + f"Expected: {expected_status_code}, Got: {actual_status_code}" + Style.RESET_ALL)
                print(f"Direct test response: {table_response.json()['message']}")
            
            if expected_response_type is not None:
                try:
                    self.assertIsInstance(table_response.json(), expected_response_type)
                    print(Fore.GREEN + f"PASS: Table '{table_name}' response type matches expected." + Style.RESET_ALL)
                except AssertionError:
                    print(Fore.RED + f"FAIL: Table '{table_name}' response type does not match expected." + Style.RESET_ALL)
                    print(Fore.YELLOW + f"Expected: {expected_response_type}, Got: {type(table_response.json())}" + Style.RESET_ALL)

            if count is not None:
                try:
                    total_count = count.split('/')[-1]
                    total_count = int(total_count)
                    self.assertEqual(total_count, expected_length)
                    print(Fore.GREEN + f"PASS: Table '{table_name}' total count matches expected." + Style.RESET_ALL)
                except AssertionError:
                    print(Fore.RED + f"FAIL: Table '{table_name}' total count does not match expected." + Style.RESET_ALL)
                    print(Fore.YELLOW + f"Expected: {expected_length}, Got: {total_count}" + Style.RESET_ALL)

            #print('-' * 50 + '\n')

class TestTableInsertAccessibility(unittest.TestCase):
    def test_table_insert_accessibility(self):

        for table in tablesToTest:

            #print('-' * 20 + table["tableName"] + '-' * 20)

            table_name = table["tableName"]
            url = f"{endpoint}{table_name}"

            test_headers = headers.copy()

            # Add Schema
            test_headers["Content-Profile"] = f"{table['schema']}"

            
            #print(url)
            
            # Perform a GET request
            table_response = requests.post(url, headers=test_headers, json={"id": "b917cbd2-f3f1-460c-9773-4b82ba720d5e"})
            
            # Check if the table is accessible
            actual_status_code = table_response.status_code
            expected_status_code = table["exeptInsertCode"] if table.get("exeptInsertCode", None) is not None else 200
            
            
            try:
                self.assertEqual(actual_status_code, expected_status_code)
                print(Fore.GREEN + f"PASS: Table '{table_name}' status code matches expected." + Style.RESET_ALL)
            except AssertionError:
                print(Fore.RED + f"FAIL: Table '{table_name}' status code does not match expected." + Style.RESET_ALL)
                print(Fore.YELLOW + f"Expected: {expected_status_code}, Got: {actual_status_code}" + Style.RESET_ALL)
                print(f"Direct test response: {table_response.json()['message']}")
            
            #print('-' * 50 + '\n')

# Run the tests
if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

.

PASS: Table 'notExistingTable' status code matches expected.
PASS: Table 'organizations' status code matches expected.
PASS: Table 'organizations' response type matches expected.
PASS: Table 'organizations' total count matches expected.
PASS: Table 'schemas' status code matches expected.
PASS: Table 'schemas' response type matches expected.
PASS: Table 'schemas' total count matches expected.
PASS: Table 'troop' status code matches expected.
PASS: Table 'records' status code matches expected.
PASS: Table 'record_changes' status code matches expected.
PASS: Table 'edges' status code matches expected.
PASS: Table 'edges' response type matches expected.
PASS: Table 'edges' total count matches expected.
PASS: Table 'plot_landmark' status code matches expected.
PASS: Table 'plot_landmark' response type matches expected.
PASS: Table 'plot_landmark' total count matches expected.
PASS: Table 'tree_coordinates' status code matches expected.
PASS: Table 'tree_coordinates' response type matches ex

.

PASS: Table 'lookup_tree_species' status code matches expected.
PASS: Table 'lookup_tree_species' response type matches expected.
PASS: Table 'lookup_tree_species' total count matches expected.
PASS: Table 'notExistingTable' status code matches expected.
PASS: Table 'organizations' status code matches expected.
PASS: Table 'schemas' status code matches expected.
PASS: Table 'troop' status code matches expected.
PASS: Table 'records' status code matches expected.
PASS: Table 'record_changes' status code matches expected.
PASS: Table 'edges' status code matches expected.
PASS: Table 'plot_landmark' status code matches expected.
PASS: Table 'tree_coordinates' status code matches expected.
PASS: Table 'plot_coordinates' status code matches expected.
PASS: Table 'subplots_relative_position' status code matches expected.
PASS: Table 'cluster' status code matches expected.
PASS: Table 'plot' status code matches expected.


.

PASS: Table 'lookup_tree_species' status code matches expected.



----------------------------------------------------------------------
Ran 3 tests in 9.070s

OK
